In [36]:
# get_lecture_list에서는 개설강좌목록 파일을 다운로드합니다.
# insert_lecture_list에서는 excel 파일을 parse하여 DB에 집어넣습니다.

# 본 노트에서는 위 과정 전체를 다루고 있습니다.
# 수정사항이 생기면 코드 및 주석을 꼭 수정합니다.

import time
from selenium import webdriver
from selenium.webdriver.common.by import By

import os
import pandas as pd
import pymysql as db
import numpy as np
from datetime import datetime

In [37]:
# 파일 다운로드 및 삽입에 사용하는 정보이므로, 미사용 시 항상 공백으로 초기화합니다.
kumoh_id = ''
kumoh_pw = ''
file_path = ''

db_user = ''
db_pw = ''
db_host = ''

In [38]:
# 사용 중인 chrome 버전에 적합한 버전의 chrome driver를 사용합니다(현재 Chrome 109 Driver).
# driver = webdriver.Chrome()

# 장치 오류가 발생하는 경우에는 아래 옵션을 추가하여 드라이버를 지정합니다.
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)

# 페이지 로드 후 요청이 이루어져야 하므로 대기 시간을 설정합니다.
driver.implicitly_wait(3)

In [39]:
driver.get("https://www.kumoh.ac.kr/_common/login/login.do?Return_Url=https://onestop.kumoh.ac.kr")
driver.implicitly_wait(2)

# 이전에는 driver.find_element_by_xpath를 사용했으나, chrome 109 driver 기준 deprecated 함수이므로 주의합니다.
driver.find_element(By.XPATH, '/html/body/div/section/div/div/div/div[1]/form/fieldset/div/div[1]/input').send_keys(kumoh_id)
driver.find_element(By.XPATH, '/html/body/div/section/div/div/div/div[1]/form/fieldset/div/div[2]/input').send_keys(kumoh_pw)
driver.implicitly_wait(1)

driver.find_element(By.XPATH, '/html/body/div/section/div/div/div/div[1]/form/fieldset/div/input').click()
time.sleep(10)

In [40]:
# 메인을 기준으로 왼쪽 패널로 이동 -> 학사관리 -> 수강신청 -> 개설강좌
driver.switch_to.default_content()
driver.switch_to.frame("LeftFrame")
time.sleep(1)

driver.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/div[3]/div[1]/div/table/tbody/tr[4]/td[2]').click()
time.sleep(1)

driver.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/div[3]/div[1]/div/table/tbody/tr[14]/td[3]').click()
time.sleep(1)

driver.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/div[3]/div[1]/div/table/tbody/tr[19]/td[4]').click()

In [41]:
# 메인 패널로 이동 -> 개설강좌 편람의 "조회" -> "엑셀"
driver.switch_to.default_content()
driver.switch_to.frame("w_cre_s9241")
time.sleep(1)

driver.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/div[2]/div[6]/table/tbody/tr/td[1]').click()
time.sleep(5)

driver.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/div[2]/div[6]/table/tbody/tr/td[2]/div/input').click()
# "엑셀" 클릭 후 대기 시간이 없으면 파일 다운로드가 정상적으로 이루어지지 않을 수 있습니다.
time.sleep(3)

In [42]:
# each_file_path_and_gen_time으로 폴더의 각 file의 경로와, 생성 시간을 저장합니다.
each_file_path_and_gen_time = []

for each_file_name in os.listdir(file_path):
    # 다운로드된 파일은 엑셀(.xlsx)이므로 엑셀이 아닌 파일은 리스트에 저장하지 않습니다.
    if ".xlsx" not in each_file_name:
        continue

    each_file_path = file_path + each_file_name
    each_file_gen_time = os.path.getctime(each_file_path)
    each_file_path_and_gen_time.append(
        (each_file_path, each_file_gen_time)
    )

# 가장 생성 시각이 큰(가장 최근인) 파일을 리턴합니다.
latest_file = max(each_file_path_and_gen_time, key=lambda x: x[1])[0]
raw = pd.read_excel(latest_file)

In [43]:
# DB에 데이터를 넣을 수 있도록 테이블과 이름을 맞춰줍니다.
df = raw[['교과목 종류', '교육과정명', '이수 대상 학년', '이수 구분',
        '교과목명', '학점', '교과목코드',
        '담당교수', '수강학과', '강의시간(강의실)',
        '제한 인원', '수강 인원', '수강 꾸러미']]
df.columns = ['교과목_종류', '교육과정명',
            '이수_대상_학년', '이수_구분',
            '교과목명', '학점', '개설교과목코드',
            '담당교수', '수강학과', '강의시간강의실',
            '제한_인원', '수강_인원', '수강_꾸러미']

# 다가오는 학기의 연도 및 학기를 지정하여 DB에 개설 강좌의 연도와 학기를 함께 저장할 수 있도록 합니다.
year = datetime.today().year
if datetime.today().month > 6:
    semester = "2"
else:
    semester = "1"

df = df.assign(년도=year)
df = df.assign(학기=semester)
df = df.replace(np.nan, '', regex=True)

In [44]:
# 금오사이 어플 내 시간표에 정보를 제공하는 DB table은 xe_home, xe_kumohtime입니다.
con = db.connect(
    user=db_user, 
    passwd=db_pw, 
    host=db_host, 
    port=3306,
    db='xe_home', 
    charset='utf8'
)
query = 'INSERT INTO xe_kumohtime(교과목_종류, 교육과정명, 이수_대상_학년, 이수_구분, 교과목명, 학점, 개설교과목코드, 담당교수, 수강학과, 강의시간강의실, 제한_인원, 수강_인원, 수강_꾸러미, 년도, 학기) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

curs = con.cursor(db.cursors.DictCursor)
for i in df.index:
    curs.execute(query, (df["교과목_종류"][i], df["교육과정명"][i], df["이수_대상_학년"][i], df["이수_구분"][i], df["교과목명"][i], df["학점"][i], df["개설교과목코드"][i], df["담당교수"][i], df["수강학과"][i], df["강의시간강의실"][i], df["제한_인원"][i], df["수강_인원"][i], df["수강_꾸러미"][i], df["년도"][i], df["학기"][i]))

con.commit()
con.close()